In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import gzip

In [2]:
import sys
sys.path.append("/Users/casey/git/mumodo/packages")

In [3]:
from mumodo import mumodoIO as mm

In [11]:
import os

folders = os.listdir('../cleaned')

In [12]:
data = {}

for f in folders:
    pid = f.split("_")[0]
    sf = mm.open_streamframe_from_xiofile(os.path.join('../cleaned/', f, 'log.xio.gz'), '/InproTK/', window_size=-1)
    ph = sf['PHASE']
    sf = sf['TASK']
    ph = ph.dropna()
    sf = sf.dropna()
    data[pid] = sf
    data[pid+'_phase'] = ph
    

opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing
opening compressed file ...
opening file without indexing


In [6]:
def get_task_frame_pairs2(sf):
    f = sf[sf.str.contains("frame")]
    frames = pd.DataFrame(f.values , columns = ['frames'])
    frames.index = f.index
    t = sf[sf.str.contains("task")]
    tasks = pd.DataFrame(t.values, columns = ['tasks'])
    tasks.index = t.index
    d = sf[sf.str.contains("domain")]
    domains = pd.DataFrame(d.values, columns = ['domains'])
    domains.index = d.index
    merged = pd.merge(frames, domains, how="outer", left_index=True, right_index=True)
    merged = pd.merge(merged, tasks, how="outer", left_index=True, right_index=True)
    merged = merged.ffill()
    print merged[:10]

In [18]:
class Task:
    def __init__(self):
        self.frame = {}
        self.task = {}
        self.start = None
        self.end = None
        self.domain = None
        self.intent = None
    def elapsed(self):
        return (self.end - self.start) / 1000

In [19]:
def get_task_frame_pairs(sf):
    current = Task()
    tasks = []
    for i,row in sf.iteritems():
        row = row.split(":")
        if len(row) < 2: continue
        elif row[1] == 'intent':
            current.intent = row[2]
            current.end = i
        elif row[0] == 'frame':
            current.frame[row[1]] = row[2]
        elif row[0] == 'domain':
            tasks.append(current)
            current = Task()
            current.domain = row[1]
            current.start = i
        elif row[0] == 'task':
            current.task[row[1]] = row[2]
    return tasks

In [20]:
def get_phase_boundaries(sf):
    pass

In [21]:
ind = 7
for pid in data:
    if "phase" in pid: continue
    phases = get_phase_boundaries(data[pid+'_phase'])
    tasks = get_task_frame_pairs(data[pid])
    print tasks[ind].domain, tasks[ind].intent
    print tasks[ind].elapsed()
    print tasks[ind].task
    print tasks[ind].frame
    break

erinnerung erinnerung
17
{'message': '[zeitung holen]'}
{'message': '>>zeitung holen '}
